In [84]:
import numpy
G = SymmetricGroup(4)
e = G.exponent()

# find the correct prime p for which we can translate the problem into Z/pZ for
def find_dixon_prime(e,G):
    candidate = next_prime(ceil(2*sqrt(G.order())))
    while (candidate % e != 1):
        candidate = next_prime(candidate+1)
    return candidate
    
p = find_dixon_prime(e,G)
F.<a> = FiniteField(p) # will work over field F = Z/pZ

In [82]:
# refer to Dixon's 1967 High Speed Computation of Group Characters paper for notation and implementation details

class DixonCharacterTable:
    def structure_constants(self,G):
        # method computing the structure constants of a group, i.e. a_rst is the number of pairs (x,y) in the conjugacy classes r and s such
        # that their product is in the conjugacy class t.

        # initialise empty dictionary
        result = {(i,j):([0]*self.classes_amount) for i in range(0,self.classes_amount) for j in range(0,self.classes_amount)}
        
        for i in range(0,self.classes_amount):
            for g in self.classes[i]:
                for j in range(0,self.classes_amount):
                    for h in self.classes[j]:
                        for k in range(0,self.classes_amount):
                            if g*h in self.classes[k]:
                                result[(i,j)][k] += 1
        return result
        
    def __init__(self, G):
        # initialise important data that will be used often during the computations

        # create a list of all conjugacy classes and their amount k
        self.classes = list(G.conjugacy_classes())
        self.classes_amount = len(self.classes)
        
        # compute the sizes of their classes, then in the notation of Dixon (1967) have h_j = class_sizes[j]
        self.class_sizes = [G.order() / G.centralizer(C.representative()).order() for C in self.classes]

        # compute the inverse class for each class, in Dixon (1967) have j' = inverse_class[j]
        # this is O(n^2) if we can hash conjugacy classes, but this isn't possible in sage
        # so it is O(nk^2)
        self.inverse_class = []
        for i in range(0,self.classes_amount):
            rep = self.classes[i].representative()
            inverse = rep.inverse()
            # compute which class contains the inverse
            for idx, cls in enumerate(self.classes,1):
                if inverse in cls:
                    self.inverse_class.append(idx)
                    break

        # initialise class matrices M_1, ..., M_k
        # computing all structure constants can be done in O(n^3)
        self.structure_constants = self.structure_constants(G)
        self.class_matrices = [matrix(ZZ, self.classes_amount, self.classes_amount, lambda x,y: self.structure_constants[(i,x)][y]) for i in range(0,self.classes_amount)]
                    

In [83]:
D = DixonCharacterTable(G)
print(D.class_matrices)

[[1 0 0 0 0]
[0 6 0 0 0]
[0 0 3 0 0]
[0 0 0 8 0]
[0 0 0 0 6], [ 0  6  0  0  0]
[ 6  0  6 24  0]
[ 0  6  0  0 12]
[ 0 24  0  0 24]
[ 0  0 12 24  0], [ 0  0  3  0  0]
[ 0  6  0  0 12]
[ 3  0  6  0  0]
[ 0  0  0 24  0]
[ 0 12  0  0  6], [ 0  0  0  8  0]
[ 0 24  0  0 24]
[ 0  0  0 24  0]
[ 8  0 24 32  0]
[ 0 24  0  0 24], [ 0  0  0  0  6]
[ 0  0 12 24  0]
[ 0 12  0  0  6]
[ 0 24  0  0 24]
[ 6  0  6 24  0]]
